In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from datetime import datetime,timedelta  
import matplotlib.pyplot as plt
import time
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline

/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
#New add
#deviceid_train.rename({'device_id':'device'}, axis=1, inplace=True)

from tiny.lda import *

from  tiny.util import *
deviceid_train = get_lda_feature()
deviceid_train = extend_feature(version='1',span_no=4, input=deviceid_train)

#print(len(deviceid_train))
#deviceid_train.groupby('max_day_cnt')['max_day_cnt'].count()
deviceid_train.head()



#deviceid_train.rename({'device':'device_id'}, axis=1, inplace=True)

2018-09-10 09:24:55,160 util_cache_file.py[60] DEBUG fn:get_lda_feature, para:[], kw:{}
2018-09-10 09:24:55,163 util_cache_file.py[21] DEBUG try to read cache from file:./cache/get_lda_feature_[]_{}.csv
/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2862: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
2018-09-10 09:24:55,412 util_cache_file.py[28] DEBUG Return 72727 resut from file cache:./cache/get_lda_feature_[]_{}.csv
2018-09-10 09:24:55,414 util_log.py[29] INFO Begin to run extend_percent with:[], {}
2018-09-10 09:24:55,417 util_cache_file.py[60] DEBUG fn:extend_percent, para:['1', 4], kw:{}
2018-09-10 09:24:55,421 util_cache_file.py[21] DEBUG try to read cache from file:./cache/extend_percent_['1', 4]_{}.csv
2018-09-10 09:24:55,907 util_cache_file.py[28] DEBUG Return 72727 resut from file cache:./cache/extend_percent_['1', 4]_{}.

,age,device,sex,app_lenghth,tfidf_sum,0,1,2,3,4,...,max_span_3,max_pkg_cnt,max_day_cnt,max_total,max_span_0_p,max_span_1_p,max_span_2_p,max_span_3_p,brand,phone_type
0,3.0,bd86d59afa24a839ce6029d718accb19,1.0,16,3.457704,0.089292,0.011770,0.808878,0.011860,0.078200,...,0.323186,9,7,1.401613,0.254,0.278,0.238,0.231,1064.0,2840.0
1,5.0,e7d158c9a8262a35c9cc630a15a9103e,1.0,12,3.037932,0.015437,0.092414,0.646381,0.015462,0.230306,...,0.407785,11,7,0.822657,0.073,0.170,0.262,0.496,420.0,1039.0
2,5.0,97abdc3828448b5acc7428dd307bc635,2.0,3,1.538901,0.050341,0.050000,0.798611,0.050298,0.050751,...,0.000000,3,6,0.087941,0.961,0.032,0.007,0.000,1054.0,2201.0
3,4.0,e4dbdbf07c9cff03d79f4872e65742b4,1.0,5,2.221036,0.034773,0.033333,0.863134,0.034388,0.034372,...,0.247198,3,7,1.076142,0.192,0.241,0.338,0.230,420.0,775.0
4,3.0,6bd4537b2970c5c6ab765c1860b88aa5,1.0,21,4.114661,0.102307,0.009092,0.278817,0.375586,0.234198,...,1.249248,8,7,4.425860,0.189,0.267,0.262,0.282,420.0,1887.0


In [5]:
# col_drop = [item for item in deviceid_train.columns if '_p' in str(item)]
# deviceid_train.drop(columns=col_drop, inplace=True )

col_drop = [item for item in deviceid_train.columns if 'max_' in str(item)]
deviceid_train.drop(columns=col_drop, inplace=True )

#deviceid_train.drop(columns=['tfidf_sum'], inplace=True )
deviceid_train.head()


,age,device,sex,app_lenghth,tfidf_sum,0,1,2,3,4,...,total_span_3,total_pkg_cnt,total_day_cnt,total_total,total_span_0_p,total_span_1_p,total_span_2_p,total_span_3_p,brand,phone_type
0,3.0,bd86d59afa24a839ce6029d718accb19,1.0,16,3.457704,0.089292,0.011770,0.808878,0.011860,0.078200,...,1.438516,16,31,5.328960,0.181,0.216,0.332,0.270,1064.0,2840.0
1,5.0,e7d158c9a8262a35c9cc630a15a9103e,1.0,12,3.037932,0.015437,0.092414,0.646381,0.015462,0.230306,...,1.957793,12,30,4.137875,0.117,0.179,0.231,0.473,420.0,1039.0
2,5.0,97abdc3828448b5acc7428dd307bc635,2.0,3,1.538901,0.050341,0.050000,0.798611,0.050298,0.050751,...,0.007136,3,22,0.367474,0.827,0.060,0.093,0.019,1054.0,2201.0
3,4.0,e4dbdbf07c9cff03d79f4872e65742b4,1.0,5,2.221036,0.034773,0.033333,0.863134,0.034388,0.034372,...,1.483204,5,21,5.130300,0.192,0.247,0.272,0.289,420.0,775.0
4,3.0,6bd4537b2970c5c6ab765c1860b88aa5,1.0,21,4.114661,0.102307,0.009092,0.278817,0.375586,0.234198,...,3.353273,21,28,11.780349,0.211,0.225,0.280,0.285,420.0,1887.0


In [6]:
train=deviceid_train[deviceid_train['sex'].notnull()]
test=deviceid_train[deviceid_train['sex'].isnull()]

In [7]:
X=train.drop(['sex','age','sex_age','device'],axis=1)
Y=train['sex_age']
Y_CAT=pd.Categorical(Y)
X_train,X_test, y_train, y_test =train_test_split(X,Y_CAT.labels,test_size=0.3, random_state=666)
lgb_train=lgb.Dataset(X_train,label=y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'max_depth':3,
    'metric': {'multi_logloss'},
    'num_class':22,
    'objective':'multiclass',
    'random_state':666,
    
}

/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  after removing the cwd from sys.path.


In [8]:
gbm = lgb.train(params,
        lgb_train,
        num_boost_round=1000,
        valid_sets=lgb_eval,
        early_stopping_rounds=300)

[1]	valid_0's multi_logloss: 3.05512
Training until validation scores don't improve for 300 rounds.
[2]	valid_0's multi_logloss: 3.02402
[3]	valid_0's multi_logloss: 2.99715
[4]	valid_0's multi_logloss: 2.97331
[5]	valid_0's multi_logloss: 2.95204
[6]	valid_0's multi_logloss: 2.93305
[7]	valid_0's multi_logloss: 2.91623
[8]	valid_0's multi_logloss: 2.90119
[9]	valid_0's multi_logloss: 2.88725
[10]	valid_0's multi_logloss: 2.87462
[11]	valid_0's multi_logloss: 2.86333
[12]	valid_0's multi_logloss: 2.85276
[13]	valid_0's multi_logloss: 2.84318
[14]	valid_0's multi_logloss: 2.8344
[15]	valid_0's multi_logloss: 2.8263
[16]	valid_0's multi_logloss: 2.81896
[17]	valid_0's multi_logloss: 2.81217
[18]	valid_0's multi_logloss: 2.80589
[19]	valid_0's multi_logloss: 2.80013
[20]	valid_0's multi_logloss: 2.79487
[21]	valid_0's multi_logloss: 2.79012
[22]	valid_0's multi_logloss: 2.78562
[23]	valid_0's multi_logloss: 2.78145
[24]	valid_0's multi_logloss: 2.77743
[25]	valid_0's multi_logloss: 2.7736

[216]	valid_0's multi_logloss: 2.72998
[217]	valid_0's multi_logloss: 2.73007
[218]	valid_0's multi_logloss: 2.7302
[219]	valid_0's multi_logloss: 2.7303
[220]	valid_0's multi_logloss: 2.73046
[221]	valid_0's multi_logloss: 2.73055
[222]	valid_0's multi_logloss: 2.7306
[223]	valid_0's multi_logloss: 2.73075
[224]	valid_0's multi_logloss: 2.73085
[225]	valid_0's multi_logloss: 2.73087
[226]	valid_0's multi_logloss: 2.73101
[227]	valid_0's multi_logloss: 2.73105
[228]	valid_0's multi_logloss: 2.73121
[229]	valid_0's multi_logloss: 2.73133
[230]	valid_0's multi_logloss: 2.73144
[231]	valid_0's multi_logloss: 2.7316
[232]	valid_0's multi_logloss: 2.7317
[233]	valid_0's multi_logloss: 2.73194
[234]	valid_0's multi_logloss: 2.73207
[235]	valid_0's multi_logloss: 2.73214
[236]	valid_0's multi_logloss: 2.7323
[237]	valid_0's multi_logloss: 2.73236
[238]	valid_0's multi_logloss: 2.73235
[239]	valid_0's multi_logloss: 2.73238
[240]	valid_0's multi_logloss: 2.73243
[241]	valid_0's multi_logloss: 

In [9]:
pre_x=test.drop(['sex','age','sex_age','device'],axis=1)
sub=pd.DataFrame(gbm.predict(pre_x.values,num_iteration=gbm.best_iteration))

In [10]:
sub.columns=Y_CAT.categories
sub['DeviceID']=test['device'].values
sub=sub[['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', '1-7','1-8', '1-9', '1-10', '2-0', '2-1', '2-2', '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']]

In [11]:
sub.to_csv('./sub/baseline.csv',index=False)